In [1]:
import os
import cv2
import glob
import time
import random

import numpy as np
import pandas as pd
from sklearn import (svm, impute, metrics, ensemble, linear_model, decomposition,
                     tree, preprocessing, model_selection)
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('../input/tabularsep21-kfolddataset/train_10folds.csv')
df_train.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f112,f113,f114,f115,f116,f117,f118,claim,nan_count,kfold
0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,86.489,...,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1,1,3
1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,...,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0,0,8
2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,15827.000,...,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1,5,6
3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,-36.837,...,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1,2,6
4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,144.120,...,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,10002.0,0.23049,1,8,9


In [3]:
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv', index_col=False)
test.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
0,957919,0.165850,0.487050,1295.00,0.02310,0.31900,0.90188,573.29,3743.7,2.705700e+12,...,0.16253,-22.1890,2.0655,0.430880,-10.7410,81606.0,1.1940,1.980400e+14,2017.1,0.46357
1,957920,0.129650,0.373480,1763.00,0.72884,0.33247,-1.26310,875.55,554370.0,5.955700e+14,...,0.81528,-1.6342,1.5736,-1.071200,11.8320,90114.0,1.1507,4.388000e+16,6638.9,0.28125
2,957921,0.120190,0.445210,736.26,0.04615,0.29605,0.31665,2659.50,317140.0,3.977800e+14,...,0.81831,-32.7800,2.1364,-1.931200,-3.2804,37739.0,1.1548,1.718100e+14,5844.0,0.13797
3,957922,0.054008,0.395960,996.14,0.85934,0.36678,-0.17060,386.56,325680.0,-3.432200e+13,...,0.86559,-2.4162,1.5199,-0.011633,1.3840,26849.0,1.1490,2.138800e+17,6173.3,0.32910
4,957923,0.079947,-0.006919,10574.00,0.34845,0.45008,-1.84200,3027.00,428150.0,9.291500e+11,...,0.25190,-18.6300,3.7387,0.757080,-4.9405,50336.0,1.2488,2.151300e+17,2250.1,0.33796


In [4]:
features = [c for c in df_train.columns if c not in ('claim', 'kfold')]
len(features)

119

In [5]:
test['nan_count'] = test.isnull().sum(axis=1)
test.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,nan_count
0,957919,0.165850,0.487050,1295.00,0.02310,0.31900,0.90188,573.29,3743.7,2.705700e+12,...,-22.1890,2.0655,0.430880,-10.7410,81606.0,1.1940,1.980400e+14,2017.1,0.46357,1
1,957920,0.129650,0.373480,1763.00,0.72884,0.33247,-1.26310,875.55,554370.0,5.955700e+14,...,-1.6342,1.5736,-1.071200,11.8320,90114.0,1.1507,4.388000e+16,6638.9,0.28125,0
2,957921,0.120190,0.445210,736.26,0.04615,0.29605,0.31665,2659.50,317140.0,3.977800e+14,...,-32.7800,2.1364,-1.931200,-3.2804,37739.0,1.1548,1.718100e+14,5844.0,0.13797,1
3,957922,0.054008,0.395960,996.14,0.85934,0.36678,-0.17060,386.56,325680.0,-3.432200e+13,...,-2.4162,1.5199,-0.011633,1.3840,26849.0,1.1490,2.138800e+17,6173.3,0.32910,0
4,957923,0.079947,-0.006919,10574.00,0.34845,0.45008,-1.84200,3027.00,428150.0,9.291500e+11,...,-18.6300,3.7387,0.757080,-4.9405,50336.0,1.2488,2.151300e+17,2250.1,0.33796,0


In [6]:
df_train.isnull().sum() # I already filled nan with modes when preparing kfold data

f1           0
f2           0
f3           0
f4           0
f5           0
            ..
f117         0
f118         0
claim        0
nan_count    0
kfold        0
Length: 121, dtype: int64

In [7]:
modes = df_train[features].mode().iloc[0]
test = test.fillna(modes)
test.isnull().sum()

id           0
f1           0
f2           0
f3           0
f4           0
            ..
f115         0
f116         0
f117         0
f118         0
nan_count    0
Length: 120, dtype: int64

In [8]:
df_train['row_min'] = df_train.min(axis=1)
df_train['row_max'] = df_train.max(axis=1)
df_train['row_mean'] = df_train.mean(axis=1)
df_train['row_std'] = df_train.std(axis=1)

test['row_min'] = test.min(axis=1)
test['row_max'] = test.max(axis=1)
test['row_mean'] = test.mean(axis=1)
test['row_std'] = test.std(axis=1)

features += ['row_min', 'row_max', 'row_mean', 'row_std']
len(features)

123

In [9]:
test.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f114,f115,f116,f117,f118,nan_count,row_min,row_max,row_mean,row_std
0,957919,0.165850,0.487050,1295.00,0.02310,0.31900,0.90188,573.29,3743.7,2.705700e+12,...,81606.0,1.1940,1.980400e+14,2017.1,0.46357,1,-2.218900e+01,6.046200e+16,9.953148e+14,7.677184e+15
1,957920,0.129650,0.373480,1763.00,0.72884,0.33247,-1.26310,875.55,554370.0,5.955700e+14,...,90114.0,1.1507,4.388000e+16,6638.9,0.28125,0,-1.569100e+01,4.388000e+16,7.804320e+14,5.597156e+15
2,957921,0.120190,0.445210,736.26,0.04615,0.29605,0.31665,2659.50,317140.0,3.977800e+14,...,37739.0,1.1548,1.718100e+14,5844.0,0.13797,1,-5.953100e+11,3.977800e+14,8.657956e+12,5.305980e+13
3,957922,0.054008,0.395960,996.14,0.85934,0.36678,-0.17060,386.56,325680.0,-3.432200e+13,...,26849.0,1.1490,2.138800e+17,6173.3,0.32910,0,-3.432200e+13,2.138800e+17,3.867230e+15,2.735275e+16
4,957923,0.079947,-0.006919,10574.00,0.34845,0.45008,-1.84200,3027.00,428150.0,9.291500e+11,...,50336.0,1.2488,2.151300e+17,2250.1,0.33796,0,-3.945700e+08,2.151300e+17,3.531646e+15,2.731726e+16


In [10]:
df_train.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f116,f117,f118,claim,nan_count,kfold,row_min,row_max,row_mean,row_std
0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,86.489,...,8.613400e+14,140.1,1.01770,1,1,3,-3.756600e+01,1.090600e+16,1.939308e+14,1.381534e+15
1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,...,7.575100e+15,1861.0,0.28359,0,0,8,-5.675800e+01,3.079400e+16,6.226979e+14,3.964814e+15
2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,15827.000,...,3.218100e+14,3838.2,0.40690,1,5,6,-4.453500e+13,3.218100e+14,5.138783e+12,4.146131e+13
3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,-36.837,...,4.533000e+12,4889.1,0.51486,1,2,6,-9.299200e+13,1.894700e+15,3.041983e+13,2.401205e+14
4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,144.120,...,-8.991300e+12,10002.0,0.23049,1,8,9,-8.991300e+12,1.012500e+16,1.849154e+14,1.290326e+15


In [11]:
scaler = preprocessing.StandardScaler()
xtest = scaler.fit_transform(test[features])

In [12]:
df_train.shape, xtest.shape

((957919, 125), (493474, 123))

In [13]:
def cross_validate_model(model, best_params, train, test):
    valid_preds = []
    true_yvalid = []
    test_preds = np.zeros((xtest.shape[0]))
    folds = 2
    for fold in range(folds):
        xtrain = train[train.kfold != fold].reset_index(drop=True)
        xvalid = train[train.kfold == fold].reset_index(drop=True)
        
        ytrain = xtrain.claim
        yvalid = xvalid.claim
        
        xtrain = scaler.transform(xtrain[features])
        xvalid = scaler.transform(xvalid[features])
        
        classifier = model(**best_params)
        classifier = classifier.fit(xtrain, ytrain)
        preds = classifier.predict_proba(xvalid)[:,1]
        
        valid_preds.extend(preds)
        true_yvalid.extend(yvalid)
        
        test_preds += classifier.predict_proba(xtest)[:, 1]
        
        roc_score = roc_auc_score(yvalid, preds)
        print(f'ROC AUC score: {roc_score}')
        
    total_roc_score = roc_auc_score(valid_preds, true_yvalid)
    print(f'Overall ROC AUC score is: {total_roc_score}')
    
    return valid_preds, test_preds/folds #for 10 fold

In [14]:
xgb_params = {
    'n_estimators' : 3600,
    'reg_lambda' : 3,
    'reg_alpha' : 26,
    'subsample' : 0.6000000000000001,
    'colsample_bytree' : 0.6000000000000001,
    'max_depth' : 9,
    'min_child_weight' : 5,
    'gamma' : 13.054739572819486,
    'learning_rate': 0.01,
    'tree_method': 'gpu_hist',
    'booster': 'gbtree'
}



In [ ]:
valid_preds = []
true_yvalid = []
test_preds = np.zeros((xtest.shape[0]))
folds = 2
for fold in range(folds):
    xtrain = df_train[df_train.kfold != fold].reset_index(drop=True)
    xvalid = df_train[df_train.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.claim
    yvalid = xvalid.claim

    xtrain = scaler.transform(xtrain[features])
    xvalid = scaler.transform(xvalid[features])

    model = XGBClassifier(**xgb_params)
    model.fit(xtrain, ytrain)
    preds = model.predict_proba(xvalid)[:,1]

    valid_preds.extend(preds)
    true_yvalid.extend(yvalid)

    test_preds += model.predict_proba(xtest)[:, 1]

    roc_score = roc_auc_score(yvalid, preds)
    print(f'ROC AUC score: {roc_score}')

total_roc_score = roc_auc_score(valid_preds, true_yvalid)
print(f'Overall ROC AUC score is: {total_roc_score}')

[05:26:23] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC AUC score: 0.8139370812763445
[05:27:50] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
valid_pred1 = pd.DataFrame()
test_pred1 = pd.DataFrame()

In [ ]:
valid_preds, test_preds = cross_validate_model(XGBClassifier,
                                              xgb_params,
                                              df_train,
                                              test)
valid_pred1['xgb'] = valid_preds
test_pred1['xgb'] = test_preds